In [56]:
# 目的是跑分，需要把 llama predict 出来的算个分

import os
import json

valid_path = "output/hf-eval-data-v3-valid"
folder_path = "output/hf-eval-data-v3-result-34b"
eval_path = "output/hf-eval-data-v3-reuslt-34b-eval"

def iter_files(folder_path):
    # 遍历文件列表
    for file in os.listdir(folder_path):
        # 获取文件的完整路径
        file_path = os.path.join(folder_path, file)

        # 判断是否为文件（而不是文件夹）
        if os.path.isfile(file_path):
            yield file_path

In [57]:
all_predict_result = []

for file in iter_files(folder_path):
    with open(file) as f:
        try:
            d = json.loads(f.read())
            all_predict_result.append(d)
        except:
            pass
    
len(all_predict_result), all_predict_result[0]

(241,
 {'path': 'output/hf-eval-data-v3-valid/f00002_extract_medical_term_relationships.py',
  'content': '# function_import --------------------\n\nfrom transformers import AutoTokenizer, AutoModel\nimport torch\n\n# function_code --------------------\n\ndef extract_medical_term_relationships(medical_term):\n    """\n    This function uses the pretrained model \'GanjinZero/UMLSBert_ENG\' from Hugging Face Transformers to find relationships between medical terms.\n    It converts the medical terms into embeddings (dense vectors) which can be compared to find similarities and relationships.\n\n    Args:\n        medical_term (str): The medical term to be converted into an embedding.\n\n    Returns:\n        torch.Tensor: The embedding of the input medical term.\n    """\n    tokenizer = AutoTokenizer.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\n    model = AutoModel.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\n\n    inputs = tokenizer(medical_term, return_tensors=\'pt\')\n    outputs 

In [58]:
# pip file + predict file => eval file
import shutil

try:
    os.mkdir(eval_path)
except:
    pass

for d in all_predict_result:
    print(d['path'])
    
    predict_file = d['path']
    
    # pip file
    pip_file = predict_file.split(".")[0] + ".txt"
    file_name = pip_file.split("/")[-1]
    
    try:
        shutil.copy(pip_file, eval_path + "/" + file_name)
    except:
        pass
    
    # eval python file
    eval_file = d['instruct'] + d['prediction'].split("\ndef")[0] + "\n\n" + d['test_function_code'] + "\n" + d['call_test_function_code']
    
    eval_file_path = eval_path + "/" + predict_file.split("/")[-1]
    
    print(eval_file_path)
    
    with open(eval_file_path, 'w') as fw:
        fw.write(eval_file)

output/hf-eval-data-v3-valid/f00002_extract_medical_term_relationships.py
output/hf-eval-data-v3-reuslt-34b-eval/f00002_extract_medical_term_relationships.py
output/hf-eval-data-v3-valid/f00004_extract_sentence_embeddings.py
output/hf-eval-data-v3-reuslt-34b-eval/f00004_extract_sentence_embeddings.py
output/hf-eval-data-v3-valid/f00016_load_graphormer_model.py
output/hf-eval-data-v3-reuslt-34b-eval/f00016_load_graphormer_model.py
output/hf-eval-data-v3-valid/f00018_estimate_image_depth.py
output/hf-eval-data-v3-reuslt-34b-eval/f00018_estimate_image_depth.py
output/hf-eval-data-v3-valid/f00020_classify_image.py
output/hf-eval-data-v3-reuslt-34b-eval/f00020_classify_image.py
output/hf-eval-data-v3-valid/f00035_sentiment_analysis.py
output/hf-eval-data-v3-reuslt-34b-eval/f00035_sentiment_analysis.py
output/hf-eval-data-v3-valid/f00049_get_legal_answer.py
output/hf-eval-data-v3-reuslt-34b-eval/f00049_get_legal_answer.py
output/hf-eval-data-v3-valid/f00051_get_answer_from_document.py
output

In [ ]:
for idx, d in enumerate(hf_eval_data):
    print(idx + 1, end="...")
    formatted_number = str(idx + 1).zfill(5)

    # check err
    matching_files_err = glob.glob(f"{output_dir}/f{formatted_number}_*.err")
    if not matching_files_err:
        continue
    
    matched += 1
        
    with open(matching_files_err[0]) as f:
        content = f.read()
        if 'Error' in content:
            continue

    # check out
    matching_files_out = glob.glob(f"{output_dir}/f{formatted_number}_*.out")
    if not matching_files_out:
        continue
        
    with open(matching_files_out[0]) as f:
        content = f.read()
        if 'failed' in content:
            continue
            
    # copy files
    matching_files = glob.glob(f"{output_dir}/f{formatted_number}_*")
    for source_file in matching_files:
        dest_file = valid_dir + source_file.split(f"{output_dir}")[-1]
        shutil.copy(source_file, dest_file)
        
    valid += 1
    
valid, matched, valid/matched

In [22]:
# get eval pass rate
import os

for i in [7, 13, 34]:
    eval_dir = f"output/hf-eval-data-v3-reuslt-{i}b-eval/"
    total_case = 0
    pass_case = 0

    for file in os.listdir(eval_dir):
        if file.endswith(".err"):
            # print(file)

            total_case += 1

            # check err file
            err_file = file
            with open(eval_dir + err_file) as f:
                content = f.read()
                if 'Error' in content:
                    continue

            # check out file
            out_file = file.split(".")[0] + ".out"
            with open(eval_dir + out_file) as f:
                content = f.read()
                if 'failed' in content:
                    continue
            
            # print(file)
            pass_case += 1

    print(f"{i}b", pass_case, total_case, pass_case / total_case)

7b 44 241 0.1825726141078838
13b 51 239 0.21338912133891214
34b 51 241 0.21161825726141079
